In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("Traffic.ipynb")

# Final Project: Traffic
## Due Date: Monday, December 13th, 11:59 PM
## Collaboration Policy

Data science is a collaborative activity. While you may talk with other groups about
the project, we ask that you **write your solutions within your own groups**. If you do
discuss the assignments with others outside of your group please **include their names** at the top
of your notebook.

# Data 100 Final Project: Traffic in a post-lockdown world

**Scenario:** You're a data scientist at Uber -- sitting in a war room on March 16, 2020, 1 day after California-wide COVID lockdown measures began and the day shelter-in-place measures are announced in the bay area. The entire data science department is on fire: All of your existing traffic models have regressed *significantly*. Given the sudden change in traffic patterns (i.e., no traffic at all), the company's traffic estimates are wildly incorrect. This is a top priority for the company. Since traffic estimates are used directly for pricing strategies, this is actively costing the company millions every hour. You are tasked with fixing these models.

**Takeaways:** How do you "fix" models that have learned biases from pre-lockdown traffic? How do you train new ones, with just 24 hours of data? What sorts of data do you examine, to better understand the situation? In the midst of company-wide panic, you'll need a strong inferential acumen to lead a robust data science response. In this project, we'll walk you through a simulated war room data science effort, culminating in some strategies to fix models online, which are experiencing large distributional shifts in data.

For this project, we'll explore traffic data provided by the **Uber Movement** dataset, specifically around the start of COVID shutdowns in March 2020. Your project is structured around the following ideas:

```
1. Guided data cleaning: Clustering data spatially
    a. Load Uber traffic speeds dataset
    b. Map traffic speeds to Google Plus Codes (spatially uniform)
        i. Load node-to-gps-coordinates data
        ii. Map traffic speed to GPS coordinates
        iii. Convert GPS coordinates to plus code regions
        iv. Sanity check number of plus code regions in San Francisco
        v. Plot a histogram of the standard deviation in speed, per plus code region.
    c. Map traffic speeds to census tracts (spatially non-uniform)
        i. Download census tracts geojson
        ii. Map traffic speed to census tracts
        iii. Sanity check number of census tracts in San Francisco with data.
        iv. Plot a histogram of the standard deviation in speed, per census tract.
    d. What defines a "good" or "bad" spatial clustering?
2. Guided EDA: Understanding COVID lockdown impact on traffic
    a. How did lockdown affect average traffic speeds?
        i. Sort census tracts by average speed, pre-lockdown.
        ii. Sort census tracts by average speed, post-lockdown.
        iii. Sort census tracts by change in average speed, from pre to post lockdown.
        iv. Quantify the impact of lockdown on average speeds.
        v. Quantify the impact of pre-lockdown average speed on change in speed.
    b. What traffic areas were impacted by lockdown?
        i. Visualize heatmap of average traffic speed per census tract, pre-lockdown.
        ii. Visualize change in average daily speeds pre vs. post lockdown.
        iii. Quantify the impact of lockdown on daily speeds, spatially.
3. Open-Ended EDA: Understanding lockdown impact on traffic times
    a. Download Uber Movement (Travel Times) dataset
4. Guided Modeling: Predict traffic speed post-lockdown
    a. Predict daily traffic speed on pre-lockdown data
        i. Assemble dataset to predict daily traffic speed.
        ii. Train and evaluate linear model on pre-lockdown data.
    b. Understand failures on post-lockdown data
        i. Evaluate on post-lockdown data
        ii. Report model performance temporally
    c. "Fix" model on post-lockdown data
        i. Learn delta off of a moving bias
        ii. Does it "solve itself"? Does the pre-lockdown model predict, after the change point?
        iii. Naively retrain model with post-lockdown data
        iv. What if you just ignore the change point?
5. Open-Ended Modeling: Predicting travel times post-lockdown
```

Concepts tested: regex, pivot, join, grouping, inferential thinking

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv
import json
import os
import contextily as cx
from collections import defaultdict
import re
from typing import Callable

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from zipfile import ZipFile
zf = ZipFile('data.zip', 'r')
zf.extractall('.')

# more readable exceptions
%pip install --quiet iwut
%load_ext iwut
%wut on

# Step 1 - Guided Data Cleaning: Partitioning Data Spatially

Our hope is answer: How do we group information spatially? We'll specifically look at 2 ways of partitioning data spatially, to understand the impact of spatial partitioning strategies on our analyses:

1. Dividing the world uniformly into slices, like Google's plus codes.
2. Dividing the world according to population, using census tracts.

In this step, we'll load the following datasets that we'll need for this project:

- Daily travel times from Uber Movement data in March 2020 from San Francisco, by census tract
- Daily traffic speeds from Uber Movement data in Q1 2020 from San Francisco, between OSM nodes
- Census tracts dividing San Francisco by GPS coordinates
- Mapping from OSM nodes to GPS coordinates

There are several terms and concepts to get familiar with upfront:

- **Open Street Maps (OSM)** provides nodes (points in space, [wiki](https://wiki.openstreetmap.org/wiki/Node)) and ways (segments between nodes [wiki](https://wiki.openstreetmap.org/wiki/Way)). These IDs are used in the Uber Movement dataset to identify streets in the traffic speeds dataset.
- **Census Tracts** provided by the county of San Francisco geographically divides space according to the US 2010 Census. This is used in the Uber Movement dataset to identify regions of differing travel times.

## 1.a. Load Uber traffic speeds dataset

The dataset is located at `data/movement-speeds-daily-san-francisco-2020-3.csv`. **Load this dataset into a dataframe.**

*The original dataset from Uber was provided hourly and took up 2.1 GB on disk, which means it couldn't fit into your 1GB of RAM. You can find the dataset preparation script at `data/PrepareTrafficDataset.ipynb` which aggregated within each day, reducing the dataset to just 55MB on disk.*

*This was originally going to be question in this project, but it takes 22 minutes to run. Better yet, if you mess up, your kernel dies and you start over. We deemed it too frustrating and preprocessed the dataset to spare you the pain... but just know that this is a real-world issue!*

<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [ ]:
# Load Uber Movement (Movement Speeds) dataset into dataframe
speeds_to_nodes = pd.read_csv('data/movement-speeds-daily-san-francisco-2020-3.csv')

speeds_to_nodes

In [ ]:
grader.check("q1a")

<!-- BEGIN QUESTION -->

## 1.b. Map traffic speed to Google Plus Codes

Google Plus Codes divide up the world uniformly into rectangular slices ([link](https://maps.google.com/pluscodes/)). Let's use this to segment traffic speeds spatially. Take a moment to answer: **Is this spatial structure effective for summarizing traffic speed?** Before completing this section, substantiate your answer with examples of your expectations (e.g., we expect A to be separated from B). After completing this section, substantiate your answer with observations you've made.

<!--
BEGIN QUESTION
name: q1b
points: 2
manual: True
-->

Our expectation is that uniform rectangular slices are not the most effective spatial structure for summarizing traffic speeds, because it groups sections of geographical regions into uniform rectangles, regardless of the type of neighborhood / subregion that the rectangle includes. Commercial districts are mixed in with residential districts, as well as highways, parks, schools, and other features that heavily play into computing average traffic speeds. 

After completing the rest of the section, we find that expectations were correct, as supported by plus codes having a lower across-cluster standard deviation of within-cluster means, as compared to other spatial structures such as census tracts. This shows that, using pluscodes, the within-cluster means are relatively similar, so it's harder to tell which subregions are high speed / low speed, preventing us from making meaningful insights about the areas.

<!-- END QUESTION -->



### 1.b.i. Load Node-to-GPS-Coordinate Data

In this substep, we'll load a mapping from OSM nodes to GPS coordinates. The dataset is provided in a gzip'ed XML file from OpenStreetMaps (OSM). The mapping from OSM nodes to GPS coordinates was downloaded from https://download.bbbike.org/osm/bbbike/SanFrancisco/SanFrancisco.osm.gz. We've downloaded this for you, to avoid any issues with OSM updates.

**If** you try to load the provided `.osm` (an `.xml` in disguise) using Python's built-in XML utilities **(by uncommenting the last 2 lines in the below cell)**, you will hit an out-of-memory error, as your kernel is forced to restart.

In [ ]:
# [OSM] - Read the OSM XML and extract mapping from node ID to GPS coordinates
PATH_OSM = os.path.expanduser('data/SanFrancisco.osm')

# Runs out of memory! File itself is 430 MB, even when filtering out
# irrelevant rows, and remaining 3M rows are too expensive to parse,
# resulting in OOM

# import xml.etree.ElementTree as ET
# _tree = ET.parse(PATH_OSM)

Your above code hits a memory error, so instead, we will use our handy-dandy tool--regex--from earlier in the semester to load just the parts of the file that we need. **Given the XML snippet below, write a regex pattern to extract OSM node ID, latitude, and longitude.** (The first capture group should be node ID. The second should be latitude, and the third should be longitude.) A snippet of the XML is included below ([screenshot](https://extract.bbbike.org/extract-screenshots.html)):

```
<?xml version='1.0' encoding='UTF-8'?>
<osm version="0.6" generator="osmconvert 0.8.3">
    <bounds minlat="42.4543" minlon="-2.4761999" maxlat="42.4..."/>
    <node id="26861066" lat="42.471111" lon="-2.454722" version="..."/>
        <tag k="name" v="Camping La Playa"/>
        <tag k="tourism" v="camp_site"/>
        <tag k="operator" v="private"/>
        ...
    </node>
    <node id="34793287" lat="42.4713587" lon="-2.4510783" version="..."/>
        <tag k="created_by" v="JOSM"/>
    </node>
    <node id="34793294" lat="42.4610836" lon="-2.4303622" version="..."/>
    <node id="34793297" lat="42.4548363" lon="-2.4287657" version="..."/>
    ...
</osm>
```

In [ ]:
# [OSM] - Read the OSM XML using a regex operation instead.
def read_node_lat_lon(path: str, pattern: str, line_condition: Callable):
    """
    Read the provided path line at a line. If the provided regex pattern
    has a match, return the grouped matches as items in a generator.
    
    :param path: Path to read data from
    :param pattern: Regex pattern to test against each line
    :param line_condition: function that returns if we should check regex
        against current line
    """
    with open(path) as f:
        for line in f:
            result = re.search(pattern, line)
            if result is not None and line_condition(result):
                yield int(result.group(1)), float(result.group(2)), float(result.group(3))

In [ ]:
node_ids = set(speeds_to_nodes.osm_start_node_id) | set(speeds_to_nodes.osm_end_node_id)

NODE_PATTERN = r"""id=.(\d+)..lat=.(\d+\.\d+)..lon=.(\-?\d+\.\d+)"""

node_to_gps = pd.DataFrame(read_node_lat_lon(
    PATH_OSM,
    pattern=NODE_PATTERN,
    line_condition=lambda result: int(result.group(1)) in node_ids
), columns=['osm_node_id', 'Latitude', 'Longitude'])
node_to_gps

In [ ]:
grader.check("q1bi")

### 1.b.ii. Map traffic speed to GPS coordinates.

Traffic speeds are currently connected to OSM nodes. You will then use the mapping from OSM nodes to GPS coordinates, to map traffic speeds to GPS coordinates. **Link each traffic speed measurement to the GPS coordinate of its starting node.**

**Note**: For simplicity, assume each segment is associated with the node it *starts* with. 

**Hint**: Not all nodes are included in the OSM node mapping. Make sure to ignore any nodes without valid GPS coordinates.

<!--
BEGIN QUESTION
name: q1bii
points: 3
-->

In [ ]:
# Find mapping from traffic speeds to GPS coordinates
speeds_to_gps = speeds_to_nodes.merge(node_to_gps, left_on="osm_start_node_id", right_on= "osm_node_id", how= "inner")
speeds_to_gps

In [ ]:
grader.check("q1bii")

### 1.b.iii. Convert GPS coordinates to plus code regions.

Plus code regions divide up the world into uniformly-sized rectangles, which we will assume is 0.012 degrees latitudiunally and longitudinally. **For each traffic speed row, compute the plus code region it belongs to**, based on its GPS coordinates.

To do this, we suggest computing a latitudinal index `plus_latitude_idx` and a longitudinal index `plus_longitude_idx` for the plus code region each row belongs to. *Make sure these columns are integer-valued*.

**Hint**: If you're running into nans, you did 1.b.ii. incorrectly!

<!--
BEGIN QUESTION
name: q1biii
points: 3
-->

In [ ]:
... # do this however you like
speeds_to_gps['plus_latitude_idx'] = (speeds_to_gps['Latitude'] / 0.012).astype(int)
speeds_to_gps['plus_longitude_idx'] = (speeds_to_gps['Longitude'] / 0.012).astype(int)
speeds_to_gps

In [ ]:
grader.check("q1biii")

### 1.b.iv. Sanity check number of plus code regions in San Francisco.

**Compute the number of unique plus codes found in your dataset**. You're checking that the number isn't ridiculous, like 1, or 100,000 (SF is 231 sq mi, so 100k tracts would average 12 sq ft per tract).

If you followed the suggestion above, this is the number of unique `(plus_latitude_idx, plus_longitude_idx)` pairs.

<!--
BEGIN QUESTION
name: q1biv
points: 4
-->

In [ ]:
# You're expecting 276 plus codes here. Don't just type "276" 
# below to pass the autograder. The goal is to sanity check your 
# dataframe!
temp = speeds_to_gps[['plus_latitude_idx', 'plus_longitude_idx']]
num_pluscode_regions = temp.drop_duplicates().shape[0]
num_pluscode_regions

In [ ]:
grader.check("q1biv")

<!-- BEGIN QUESTION -->

### 1.b.v. How well do plus code regions summarize movement speeds?

The following will give us an idea of how well the average represents traffic speed per plus code region. For these questions, we'll refer to a "plus code region" as a "cluster":

1. **Plot a histogram of the within-cluster standard deviation**.
2. **Compute across-cluster average of within-cluster standard deviation**.
3. **Compute across-cluster standard deviation of within-cluster average speeds**.
4. **Is this average variance reasonable?** To assess what "reasonable" means, consider these questions and how to answer them: (1) Do plus codes capture meaningful subpopulations? (2) Do differences between subpopulations outweigh differences within a subpopulation? Use the statistics above to answer these questions, and compute any additional statistics you need. Additionally explain *why these questions are important to assessing the quality of a spatial clustering*.

**Hint**: Run the autograder first to ensure your variance average and average variance are correct, before starting to draw conclusions.

In the first cell, write your written answers. In the second cell, complete the code.

<!--
BEGIN QUESTION
name: q1bv1
points: 2
manual: True
-->



For plus codes, the average variance is not a reasonable statistic. 

Plus codes do not capture meaningful subpopulations because they split a region up into uniform rectangles which can contain “slices” of many different traffic zones, such as highways (high speed), school zones (low speed), commercial & residential districts, etc. This causes within-cluster means to be less meaningful, because the mixture of these zones causes each cluster’s mean to be pretty similar. 

In theory, average variance represents how much the distribution of inner-cluster means is spread out. However, this statistic is inflated because the amount of data available for each cluster isn't consistent. For clusters with a lot of data points, we would expect the mean and standard deviations of traffic speeds to be accurate representations of real world traffic in that pluscode. However, statistically, clusters with less available data are more likely to have higher within-cluster standard deviations. 

Therefore, computing the across-cluster mean of within-cluster standard deviations gives disproportionate weight to clusters with less data, because each of the clusters hold the same weight when computing an average across all clusters. This inflates the average variance figure, thereby making it a less reasonable and reliable statistic. 

This is important in assessing the quality of spatial clustering because we need quantitative metrics to see if subpopulations carry any real meaning. For instance, a clustering method with a very high average variance tells us that when you look at any particular cluster, it is likely that the traffic speeds within that cluster vary heavily. Then, we can step back and assess if this clustering method has meaningful subpopulations, as we now see that we may have grouped too many high-speed and low-speed regions together in the same clusters. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q1bv2
points: 2
manual: True
-->

In [ ]:
speed_variance_by_pluscode = speeds_to_gps.groupby(['plus_latitude_idx','plus_longitude_idx']).agg(np.std)
average_variance_by_pluscode = np.mean(speed_variance_by_pluscode['speed_mph_mean'])
variance_average_by_pluscode = speeds_to_gps.groupby(['plus_latitude_idx','plus_longitude_idx']).mean()["speed_mph_mean"].std()
plt.hist(speed_variance_by_pluscode['speed_mph_mean']);
plt.ylabel("Number of Pluscodes")
plt.xlabel("Within-Cluster Standard Deviation of Speed in mph")
plt.title("Speed Variance by Pluscode");


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q1bv3
points: 3
-->

In [ ]:
speed_variance_by_pluscode

In [ ]:
grader.check("q1bv3")

## 1.c. Map traffic speed to census tract.

Census tracts divide the space much less uniformly, subdividing regions that we were interested in into smaller zones. This suggests promise in providing informative spatial segments. Note that the daily traffic speeds are provided between OpenStreetMap (OSM) nodes, so we'll need to map nodes to census tracts somehow.

Above, we've mapped traffic speeds to GPS coordinates. Below, we'll then link GPS coordinates to census tracts, to complete the mapping from traffic speeds to census tracts.

### 1.c.i. Download Census Tracts Geojson

**Load the census tracts geojson.** Make sure to see the relevant [geopandas io documentation](https://geopandas.org/docs/user_guide/io.html) to see how to load a geojson.

**Hint**: It should take you just one line to load.

<!--
BEGIN QUESTION
name: q1ci
points: 1
-->

In [ ]:
PATH_TRACTS = os.path.expanduser('data/san_francisco_censustracts.json')
tract_to_gps = gpd.read_file(PATH_TRACTS)
tract_to_gps['MOVEMENT_ID'] = tract_to_gps['MOVEMENT_ID'].astype(int)
tract_to_gps

In [ ]:
grader.check("q1ci")

### 1.c.ii Map traffic speed to census tracts.

You will need to *spatially join* the (1) mapping from traffic speed to GPS coordinates `speed_to_gps` and (2) the mapping from GPS coordinates to boundaries of census tracts `tract_to_gps` to group all traffic speeds by census tract. This "spatial join" is an advanced feature recently released (as of time of writing, in Oct 2021) in geopandas, which allows us to connect single points to their enclosing polygons. You will do this question in 3 parts:

1. Convert the last dataframe `speeds_to_gps` into a geopandas dataframe `speeds_to_points`, where GPS coordinates are now geopandas points. See this tutorial: https://geopandas.org/gallery/create_geopandas_from_pandas.html#From-longitudes-and-latitudes
2. Set the coordinate-system for the new geopandas dataframe to the "world geodesic system" [link](https://epsg.io/4326), or in other words, the coordinate system that GPS coordinates are reported in.
3. Compute a spatial join between census tracts `tract_to_gps` and the geopandas traffic speeds `speeds_to_points`

<!--
BEGIN QUESTION
name: q1cii
points: 4
-->

In [ ]:
speeds_to_points = gpd.GeoDataFrame(speeds_to_gps, geometry=gpd.points_from_xy(speeds_to_gps.Longitude, speeds_to_gps.Latitude)).set_crs(4326)
speeds_to_tract = tract_to_gps.sjoin(speeds_to_points, how = "inner").to_crs(4326)

speeds_to_tract

In [ ]:
grader.check("q1cii")

### 1.c.iii. Aggregate movement speeds by census tract.

- Create a new dataframe `speeds_by_tract` to group movement speeds by census tract. See the outputted dataframe from 1.c.i. to check how census tracts are identified.
- Always double-check your numbers. **Report the number of census tracts** in your dataset.

<!--
BEGIN QUESTION
name: q1ciii
points: 2
-->

In [ ]:
speeds_by_tract = speeds_to_tract.groupby('MOVEMENT_ID')
num_census_tracts = len(speeds_by_tract)
num_census_tracts

In [ ]:
grader.check("q1ciii")

<!-- BEGIN QUESTION -->

### 1.c.iv. How well do census tracts summarize movement speeds?

The following will give us an idea of how well the average represents traffic speed per plus code region. For these questions, we'll refer to a "census tract" as a "cluster":

1. **Plot a histogram of the within-cluster standard deviation**.
2. **Compute across-cluster average of within-cluster standard deviation**.
3. **Compute across-cluster standard deviation of within-cluster average speeds**.
4. **Is this average variance reasonable?** To assess what "reasonable" means, consider these questions and how to answer them: (1) Do plus codes capture meaningful subpopulations? (2) Do differences between subpopulations outweigh differences within a subpopulation? Use these ideas to assess whether the average standard deviation is high or not.

Note: We are using the speed metric of miles per hour here.

Just like before, please written answers in the first cell and coding answers in the second cell.
<!--5. Using the above, how would you **compare census tracts to plus codes, in terms of its effectiveness** as a spatial clustering mechanism for analyzing traffic speeds? Compare the statistics you've computed. What does it mean for one to be higher than the other?-->

<!--
BEGIN QUESTION
name: q1civ1
points: 2
manual: True
-->



This average variance is much more reasonable than the pluscode average variance. While the calculation for both ends up being similar (8.68 vs. 8.30), we see that average variance as a statistic is much more meaningful when used on census tracts.

This is largely because census tracts are divided up to have roughly the same population in each tract, which reduces the problem we explained earlier, where clusters with low populations (number of data points) are disproportionately factored into the calculation of across-cluster average variance. In other words, because census tracts have more similar populations than pluscodes, we see that differences (deviation) within a cluster no longer outweigh the differences across all clusters, thereby making the average variance a much more reliable statistic. 

Similarly, census tracts summarize movement speeds better than plus code regions because the subregions are defined based on neighborhood type and population counts. For instance, specific suburbs are zoned together in a single census tract, seperated from commercial districts, industrial areas, etc. 

This makes each census tract more different from each other, which can be seen by the clusters' means varying more from one another. This is why we see an increase in across-cluster standard deviation of within-cluster average speeds. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q1civ2
points: 2
manual: True
-->

In [ ]:
speed_variance_by_tract = speeds_to_tract.groupby(['MOVEMENT_ID']).agg(np.std)
average_variance_by_tract = np.mean(speed_variance_by_tract['speed_mph_mean'])
variance_average_by_tract = speeds_to_tract.groupby(['MOVEMENT_ID']).mean()["speed_mph_mean"].std() 
plt.hist(speed_variance_by_tract['speed_mph_mean'])
plt.ylabel("Number of Census Tracts") 
plt.xlabel("Within-Cluster Standard Deviation of Speed in mph") 
plt.title("Speed Variance by Tract");

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q1civ3
points: 3
-->

In [ ]:
speed_variance_by_tract

In [ ]:
grader.check("q1civ3")

<!-- BEGIN QUESTION -->

## 1.d. What would be the ideal spatial clustering?

This is an active research problem in many spatiotemporal modeling communities, and there is no single agreed-upon answer. Answer both of the following specifically knowing that you'll need to analyze traffic patterns according to this spatial clustering:

1. **What is a good metric for a spatial structure?** How do we define good? Bad? What information do we expect a spatial structure to yield? Use the above parts and questions to help answer this.
2. **What would you do to optimize your own metric for success in a spatial structure?**

See related articles:

- Uber's H3 [link](https://eng.uber.com/h3/), which divides the world into hexagons
- Traffic Analysis Zones (TAZ) [link](https://en.wikipedia.org/wiki/Traffic_analysis_zone), which takes census data and additionally accounts for vehicles per household when dividing space

<!--
BEGIN QUESTION
name: q1d
points: 3
manual: True
-->

A good spatial structure is one where we can look at each cluster and generate area-specific insights, for instance looking at how traffic speeds change in different neighborhoods / subregions over time. In other words, when each cluster has a "type" (suburb, low-income or high income, commercial districts, downtown, highways, etc), we can make meaningful conclusions about how each "type" is affected by something like a lockdown. 

A bad spatial structure, in this case, is one where lines are drawn somewhat arbitrarily, grouping together different types of neighborhoods and districts in the same cluster. Lack of differentiation prevents meaningful insight across clusters. Also, a bad spatial cluster is one where the size of each cluster is too small or too big. If the clusters are too small, we don't generate much more insight than simply looking at each node. If the clusters are too large, entire districts / zones are grouped together, preventing us from making detailed insights. 

A good metric for spatial clustering would be a high variance of within-cluster means. If we see a high variance of within-cluster means, it indicates that the clusters represent different types of neighborhood / subregions, i.e. residential vs. industrial / highways. Another good metric would be across-cluster standard deviation of within-cluster number of data points. This would tell us if the amount of available data is similar across all clusters. We would likely want this variance to be low, meaning each cluster has roughly the same number of data points, allowing our calculations made across all clusters to be evenly-weighted.

Addressing the more general question, we think the ideal spatial clustering for traffic speeds would be something like census tracts, where the entire region is separated by neighborhood type. Furthermore, if each cluster had the same amount of data available, statistics such as across-cluster standard deviation and mean would be more meaningful, because each cluster would be weighed equally, giving a more accurate representation instead of low-data clusters holding disproportionate weight. 

If we were creating our own spatial structure, we would want to “draw” and “redraw” our lines, adjusting in different ways to optimize for the metrics outlined above. For instance, if our across-cluster standard deviation of within-cluster means is too low, our subregions might be too similar to one another. In this case, we might want to split the region into more clusters, allowing each one to represent a more specific part of the city.

<!-- END QUESTION -->



# Step 2 - Guided EDA: Understanding COVID Lockdown Impact on Traffic

In this step, we'll examine the impact of COVID on traffic. In particular, we'll study 3 different questions:

- How did lockdown affect traffic speed? What factors dictate how much lockdown affected traffic speed?
- What areas of traffic were most impacted by lockdown?

## 2.a. How did lockdown affect traffic speed?

<!-- BEGIN QUESTION -->

### 2.a.i. Sort census tracts by average speed, pre-lockdown.

Consider the pre-lockdown period to be March 1 - 13, before the first COVID-related restrictions (travel bans) were announced on March 14, 2020.

1. **Report a DataFrame which includes the *names* of the 10 census tracts with the lowest average speed**, along with the average speed for each tract.
2. **Report a DataFrame which includes the *names* of the 10 census tracts with the highest average speed**, along with the average speed for each tract.
2. Do these names match your expectations for low speed or high speed traffic pre-lockdown?  What relationships do you notice? (What do the low-speed areas have in common? The high-speed areas?) For this specific question, answer qualitatively. No need to quantify. **Hint**: Look up some of the names on a map, to understand where they are.
3. **Plot a histogram for all average speeds, pre-lockdown**.
4. You will notice a long tail distribution of high speed traffic. What do you think this corresponds to in San Francisco? Write down your hypothesis.

Hint: To start off, think about what joins may be useful to get the desired DataFrame.

<!--
BEGIN QUESTION
name: q2ai1
points: 3
manual: True
-->

The names of the lowest-speed census tracts definitely do match my expectation because they're all in high-congestion areas of San Francisco. As someone who grew up here, I expected neighborhoods like the Tenderloin, Mission District, and the Financial District to be very slow because a huge volume of cars and trucks occupy the streets throughout most of the day, slowing down traffic. Furthermore, many of these neighborhoods contain unusual roads and traffic conditions, such as trolleys, intersecting one-way streets, double bike lanes, looping freeway entrances, and other features that often cause confusion and slow down traffic due to uncertainty. 

The names of the high-speed census tracts also match my expectation because they're almost all near highways or in industrial parts of the Bay Area. For tracts that include highways, we would expect a much higher average traffic speed because highways have much higher speed limits. For tracts in industrial areas, such as Petrolite Street in Richmond, CA, we see many large open roads almost entirely occupied by working trucks and cars. It makes sense that these areas have high average traffic speeds because there is a lot of open space and not much residential / commercial activity. 

<!-- END QUESTION -->

Answer the following question:
<!--
BEGIN QUESTION
name: q2ai2
points: 3
-->

In [ ]:
# compute the average speed per census tract (will use this later),
# BEFORE the shelter-in-place was announced on March 14, 2020.
# Autograder expects this to be a series
averages_pre = speeds_to_tract[speeds_to_tract['day'] < 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
# Autograder expects this to be a dataframe with name of census tract,
# polygon for census tract, and average speed per census tract
averages_pre_named = speeds_to_tract[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(averages_pre.to_frame().reset_index())
averages_pre_named

In [ ]:
grader.check("q2ai2")

Report the lowest 10 census tracts with the lowest average speed
Remember we want the NAME of each census tract too. For the autograder, please keep the name of the speed field, `speed_mph_mean`.

<!--
BEGIN QUESTION
name: q2ai3
points: 1
-->

In [ ]:
bottom10_averages_pre = averages_pre_named.sort_values(by=['speed_mph_mean'])[:10]
bottom10_averages_pre

In [ ]:
grader.check("q2ai3")

Report the highest 10 census tracts with the highest average speed.

<!--
BEGIN QUESTION
name: q2ai4
points: 1
-->

In [ ]:
top10_averages_pre = averages_pre_named.sort_values(by=['speed_mph_mean'],ascending = False)[:10]
top10_averages_pre

In [ ]:
grader.check("q2ai4")

<!-- BEGIN QUESTION -->

Plot the histogram
<!--
BEGIN QUESTION
name: q2ai5
points: 1
manual: True
-->

In [ ]:
plt.hist(averages_pre)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Hisogram of Bay Area Census Tracts Average Speed Pre Lockdown");

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2.a.ii. Sort census tracts by average speed, post-lockdown.

I suggest checking the top 10 and bottom 10 tracts by average speed, post-lockdown. Consider the post-lockdown period to be March 14 - 31, after the first COVID restrictions were established on March 14, 2020. It's a healthy sanity check. For this question, you should report:

- **Plot a histogram for all average speeds, post-lockdown.**
- **What are the major differences between this post-lockdown histogram relative to the pre-lockdown histogram above**? Anything surprising? What did you expect, and what did you find?

Write the written answers in the cell below, and the coding answers in the cells after that.

<!--
BEGIN QUESTION
name: q2aii1
points: 1
manual: True
-->

The biggest difference between the histograms is that average speed went up significantly after the lockdown. Before the lockdown, the top 10 census tracts with the highest average speed ranged from (38.9 mph – 59.5 mph), averaging 46.4 mph across the 10 tracts. After the lockdown, the top 10 tracts ranged from (56.0 mph – 70.5 mph), averaging 64.9 mph across the 10 tracts.

While the shape of the distributions is similar, the average speed in the highest tracts went way up after the lockdown. This is surprising to me, because I would have expected high speeds to be associated with being in a rush to get somewhere, but almost all schools, places of work, stores, etc. were shut down. However, I can also see how if fewer total cars were out on the roads, people's driving speeds aren't limited by the density of traffic around them, which could explain higher average speeds.

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q2aii2
points: 2
-->

In [ ]:
# compute the average speed per census tract (will use this later),
# AFTER (and including) the first COVID restrictions were put into effect.
# Autograder expects this to be a series
averages_post = speeds_to_tract[speeds_to_tract['day'] >= 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
# Autograder expects this to be a dataframe with name of census tract,
# polygon for census tract, and average speed per census tract
averages_post_named = speeds_to_tract[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(averages_post.to_frame().reset_index())
averages_post_named

In [ ]:
grader.check("q2aii2")

<!-- BEGIN QUESTION -->

Plot the histogram
<!--
BEGIN QUESTION
name: q2aii3
points: 1
manual: True
-->

In [ ]:
plt.hist(averages_post)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Hisogram of Bay Area Census Tracts Average Speed Post Lockdown");

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2.a.iii. Sort census tracts by change in traffic speed from pre to post lockdown.

For each segment, compute the difference between the pre-lockdown average speed (March 1 - 13) and the post-lockdown average speed (March 14 - 31). **Plot a histogram of all differences.** Sanity check that the below histogram matches your observations of the histograms above, on your own.

<!--
BEGIN QUESTION
name: q2aiii
points: 2
manual: True
-->

In [ ]:
# The autograder expects differences to be a series object with index
# MOVEMENT_ID.
diff = averages_pre_named.merge(averages_post_named, left_on = "DISPLAY_NAME",right_on = "DISPLAY_NAME", how = 'inner')
diff['differences'] = diff['speed_mph_mean_y'] - diff['speed_mph_mean_x']
differences = diff['differences']
# plot the differences
plt.hist(differences);
plt.xlabel("Difference in Average Speed between Pre-Lockdown and Post-Lockdown") 
plt.ylabel("Number of Census Tracts")
plt.title("Differences in Average Speeds Before and After Lockdown");

In [ ]:
grader.check("q2aiii")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2.a.iv. Quantify the impact of lockdown on average speeds.

1. **Plot the average speed by day, across all segments**. Be careful not to plot the average of census tract averages instead. Recall the definition of segments from Q1.
2. Is the change in speed smooth and gradually increasing? Or increasing sharply? Why? Use your real-world knowledge of announcements and measures during that time, in your explanation. You can use this list of bay area COVID-related dataes: https://abc7news.com/timeline-of-coronavirus-us-covid-19-bay-area-sf/6047519/

<!--
BEGIN QUESTION
name: q2aiv1
points: 1
manual: True
-->

In [ ]:
# Autograder expects this to be a series object containing the
# data for your line plot -- average speeds per day.
speeds_daily = speeds_to_tract.groupby('day').agg(np.mean)['speed_mph_mean'] 
plt.plot(speeds_daily)
plt.xlabel("Day in March 2020")
plt.ylabel("Average Speed Across All Segments")
plt.title("Average Speed Across all Nodes in March 2020");

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Write your written answer in the cell below

<!--
BEGIN QUESTION
name: q2aiv2
points: 1
manual: True
-->

The change in speed is increasingly sharpy, specifically accelerating around March 17th. This is likely because on March 17, shelter in place went into effect in almost all Bay Area counties, notably San Francisco and Alameda counties being relevant to this dataset. 

Shelter in place caused most schools, places of work, stores, etc. to close down, meaning people didn't have anywhere to go. This likely caused decreases in traffic volume across the board, which would explain the sharp increase in average traffic speeds, because people's driving speeds were not limited by congestion and traffic density.

<!-- END QUESTION -->

Ignore the empty cell below, just run the autograder to test the code above is correct.
<!--
BEGIN QUESTION
name: q2aiv3
points: 1
-->

In [ ]:
grader.check("q2aiv3")

<!-- BEGIN QUESTION -->

### 2.a.v. Quantify the impact of pre-lockdown average speed on change in speed.

1. Compute the correlation between change in speed and the *pre*-lockdown average speeds. Do we expect a positive or negative correlation, given our analysis above?
2. Compute the correlation between change in speed and the post-lockdown average speeds.
3. **How does the correlation in Q1 compare with the correlation in Q2?** You should expect a significant change in correlation value. What insight does this provide about traffic?

Written answers in the first cell, coding answerts in the following cell.

<!--
BEGIN QUESTION
name: q2av1
points: 2
manual: True
-->

We would expect the correlation between change in speed and pre-lockdown average speeds to be positive, because it makes sense that roads with high speeds before the lockdown also increased in speed post-lockdown. As traffic speeds generally increased across the board, roads such as highways with already high “pre-lockdown speeds” likely freed up even more, shown by a positive “change in speed”. 

For a similar reason, we would expect the correlation between change in speed and post-lockdown average speeds to be positive, because roads with high “post-lockdown speeds” likely saw a large increase in speed as compared to pre-lockdown (positive change in speed). 

After computing both correlation coefficients, we see that both correlations are positive, as expected, but post-lockdown speed is much higher correlated with change in speed (0.79 vs. 0.46). This is interesting, as it tells us that, looking at all the roads post lockdown, slower streets did not increase very much, while higher-speed streets increased the most. 

This is likely because the roads with the highest capacity for speeds such as highways, saw the biggest increase in speeds. This makes sense because, pre-lockdown, highways might not necessarily have high average speeds, depending on how much traffic congestion they normally get. Then, when the lockdown decreased traffic congestion overall, these roads with high capacities for speed saw the biggest increase in speeds, as well as having a high post-lockdown average speed. 

In other words, the lockdown made it so traffic speeds better reflect speed limits. As such, high speed-limit roads, such as highways, saw the biggest increases in speeds because they have the capacity for high-speed traffic. Similarly, streets with low post-lockdown average speeds did not increase by much, likely because their low speeds weren’t due to traffic congestion, but instead by low speed limits, such as roads in school zones, steep hills, etc. 

All of this generates useful insights about real traffic phenomena, as we are putting together more detailed “profiles” for each road and how they are affected differently by something like a county-wide lockdown.  

<!-- END QUESTION -->


<!--
BEGIN QUESTION
name: q2av2
points: 2
-->

In [ ]:
corr_pre_diff = np.corrcoef(diff['speed_mph_mean_x'], diff['differences'])[1][0]
corr_post_diff = np.corrcoef(diff['speed_mph_mean_y'],diff['differences'])[1][0] 
corr_pre_diff, corr_post_diff

In [ ]:
grader.check("q2av2")

## 2.b. What traffic areas were impacted by lockdown?

<!-- BEGIN QUESTION -->

### 2.b.i. Visualize spatial heatmap of average traffic speed per census tract, pre-lockdown.

Visualize a spatial heatmap of the grouped average daily speeds per census tract, which you computed in previous parts. Use the geopandas [chloropleth maps](https://geopandas.org/docs/user_guide/mapping.html#choropleth-maps). **Write your observations, using your visualization, noting down at least 2 areas or patterns of interest**. These may be a local extrema, or a region that is strangely all similar.

**Hint**: Use [`to_crs`](https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) and make sure the `epsg` is using the Pseudo-Mercator projection.

**Hint**: You can use `contextily` to superimpose your chloropleth map on a real geographic map.

**Hint** You can set a lower opacity for your chloropleth map, to see what's underneath, but be aware that if you plot with too low of an opacity, the map underneath will perturb your chloropleth and meddle with your conclusions.

Written answers in the first cell, coding answers in the second cell.

<!--
BEGIN QUESTION
name: q2bi1
points: 1
manual: True
-->

The region near the SFO airport increased the most in speed. This makes sense because, after the lockdown, there weren’t many people traveling using airplanes, meaning highways and other roads near SFO became much less congested. Due to this, traffic speeds increased drastically. 

Another interesting pattern we noticed was that the average speed in downtown San Fransisco did not have a drastic change before and after the lockdown. This seemed quite strange due to the fact that downtown is considered an extremely crowded region, which made me assume that after the lockdown there should be much less people traveling downtown. I would assume that post lockdown, the speed of traffic should have changed drastically. That said, these roads might have other conditions that slow down traffic other than density of traffic. Steep hills, narrow roads, low speed limits, etc., likely kept traffic speeds low even when there wasn’t much congestion post-lockdown.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2bi2
points: 4
manual: True
-->

In [ ]:
fig, ax =plt.subplots(1, 1)
avgs_pre_heatmap = gpd.GeoDataFrame(averages_pre_named).to_crs(3857) 
avgs_pre_heatmap.plot('speed_mph_mean', ax = ax, legend = True) 
cx.add_basemap(ax, crs = avgs_pre_heatmap.crs.to_string(), source = cx.providers.Stamen.TonerLite);

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2.b.ii. Visualize change in average daily speeds pre vs. post lockdown.

Visualize a spatial heatmap of the census tract differences in average speeds, that we computed in a previous part. **Write your observations, using your visualization, noting down at least 2 areas or patterns of interest.** Some possible ideas for interesting notes: Which areas saw the most change in average speed? Which areas weren't affected? Why did some areas see *reduced* average speed?

First cell is for the written answers, second cell is for the coding answers.

<!--
BEGIN QUESTION
name: q2bii1
points: 1
manual: True
-->

One interesting pattern I noticed from the plot was that downtown San Francisco had the lowest average speed, which was close to 20 mph. This makes sense because downtown San Francisco is the most populated place with the most traffic, so the average speed during pre-lockdown should be slower than post-lockdown. 

Another observation is it looks like the area around Pacifica, the south-west most region on the heatmap, actually saw a reduced average speed after the lockdown. This could be because that area is home to a huge number of hiking trails, beaches, and other outdoor activities that seemingly saw an increase in traffic after the lockdown. This is likely because people were extremely limited in activities, and being outdoors was one of the only ways people could spend their time outside of their home.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<!--
BEGIN QUESTION
name: q2bii2
points: 4
manual: True
-->

In [ ]:
fig, ax =plt.subplots(1, 1)
speed_change_heatmap = gpd.GeoDataFrame(diff, geometry = 'geometry_x').to_crs(3857)
speed_change_heatmap.plot('differences', ax = ax, legend = True) 
cx.add_basemap(ax, crs = avgs_pre_heatmap.crs.to_string(), source = cx.providers.Stamen.TonerLite);

<!-- END QUESTION -->



# Step 3 - Open-Ended EDA: Understanding lockdown impact on travel times

Explore daily travel times from Hayes Valley to other destinations both before and throughout lockdown. Use the following questions as suggestions for what to explore, temporally and spatially:

- How did lockdown affect travel times? Are there any meaningful factors that determined how travel time would be impacted? How was travel time affected over time?
- Travel to which destinations were affected by lockdown? Are there surprisingly disproportionate amounts of impact in certain areas?

## 3.a. Load Datasets

In this step, we will load two datasets:

- Daily travel times from Hayes Valley to all other census tracts around San Francisco.
- Daily travel times from 300 Hayes St to Golden Gate Park in San Francisco.

For this specific set of data, we can ask several more questions; which questions you pursue are up to you, including any that you come up that are not on this list:

- Which routes from Hayes Valley had similar impact on travel time? Did they share any factors in common? Traveling through the same place -- e.g., a freway? Traveling in similar areas e.g., residential areas?
- Were clusters of routes impacted more severely than others over time? What determined the degree of impact?

In [ ]:
PATH_TIMES = 'data/travel-times-daily-san-francisco-2020-3.csv'
times_to_tract = pd.read_csv(PATH_TIMES)
times_to_tract

# Step 4 - Guided Modeling: Predict traffic speed post-lockdown

In this step, you'll train a model to predict traffic speed. In particular, you'll learn how to provide implicit supervision and correction to your model, when you know there's been a distribution shift in its data, leading to a large gap between train and test sets. You'll follow the following outline:

- Build a model to predict daily traffic speed in San Francisco. Train and evaluate on *pre*-lockdown traffic speeds around the city.
- Evaluate your model on post-lockdown traffic speeds. Where is your model most mistaken, and why?
- Using this knowledge, how would you correct your model for a more accurate post-lockdown traffic predictor?


The technical term for a phenomenon like the lockdown, which caused major distributional shifts in the data, is *change point*. A large body of work studies "change point detection," but you'll be harder pressed to find a "handling change point" paper. 

## 4.a. Predict daily traffic speed on pre-lockdown data

For your model, you will predict daily traffic speed per census tract, given the previous $k=5$ daily traffic speeds for that census tract. In particular, say a matrix $A$ is $n \times d$, where $n$ is the number of census tracts and $d$ is the number of days. We define the following inputs and labels:

$$X_{(i,t)} = [A_{(i,t-5)}, A_{(i,t-4)}, A_{(i,t-3)}, A_{(i,t-2)}, A_{(i,t-1)}]$$
$$y_{(i,t)} = [A_{(i,t)}]$$

This just means that each sample $X_i$ includes speed averages from the previous 5 days for the $i$th census track.

### 4.a.i. Assemble dataset to predict daily traffic speed.

Below, we've included skeletons for the helper functions we defined, to complete the problem. We highly recommend following this skeleton code, else we cannot guarantee staff support for debugging your work.


**Hint**: What's wrong with collecting all samples, then randomly selecting some percentage to hold out? See the answer in the expandable below.

<details>
    <summary>[Click to expand] How to do train-validation split correctly, on time series</summary>
    
For a *time series* in particular, this random split would be cheating, because data within each day is highly correlated. Instead, you should hold out entire days from the dataset. In this case, you should hold out the last 2 days for your validation set.
</details>

<!--
BEGIN QUESTION
name: q4ai1
points: 1
-->

In [ ]:
def dataframe_to_time_series(df: pd.DataFrame):
    """Convert your dataframe into a 'time series'.
    
    :param df: the original dataframe, mapping speeds to census tracts.
        This dataframe should contain the `MOVEMENT_ID` (census tract id),
        `day`, and average speed for that day `speed_mph_mean`
    :return: a new dataframe that is formatted as n x d, where
        n is the number of samples (census tracts) and d is the number of
        dimensions (days). The values are the speeds.
    """
    return pd.pivot_table(data = df, columns = 'day', index = 'MOVEMENT_ID', values = 'speed_mph_mean')
time_series = dataframe_to_time_series(speeds_to_tract)
time_series_pre = time_series.iloc[:, list(range(13))]

In [ ]:
grader.check("q4ai1")

In [ ]:
def time_series_to_numpy(df: pd.DataFrame, T: int, n_val: int):
    """Convert your 'time series' into train-validate splits, in numpy
    
    You can assume your dataframe contains a `day` column where days
    start from 1 and are consecutive.
    
    :param df: the dataframe formatted as n x d, where
        n is the number of samples (census tracts) and d is the number of
        dimensions (days). The values are the speeds.
    :param T: number of days to include in each training sample
    :param n_val: number of days to hold out for the validation set.
        Say we have 5 total days in our dataset, T=2, n_val=2. This means
        during training, we have samples that pull averages from days 1 and
        2 to predict day 3: x=(1, 2), y=(3,) For validation, we have samples
        like x=(2, 3), y=(4,) and x=(3, 4), y=(5,). This way, the model sees
        data from days 4 and 5 only during validation.
    :return: Set of 4 numpy arrays - X_train, y_train, X_val, y_val - where
        X_* arrays are (n, T) and y_* arrays are (n,).
    """
    N = df.shape[1]
    df_train = df.iloc[:, :N - n_val]
    df_val = df.iloc[:, N - n_val - T:N]
    first = pd.DataFrame()
    second = pd.DataFrame()
    for item in range(N - n_val - T): 
        shift = df_train.shift(-item, axis = 1).iloc[:, :T + 1]
        first = first.append(shift)
    for item in range(n_val): 
        shift = df_val.shift(-item, axis = 1).iloc[:, :T + 1]
        second = second.append(shift)
    X_train = np.array((first.iloc[:, :-1]))
    y_train = np.array(first.iloc[:, -1:].unstack())
    X_val = np.array((second.iloc[:, :-1]))
    y_val = np.array(second.iloc[:, -1:].unstack())
    return X_train, y_train, X_val, y_val

def remove_nans(X: np.array, y: np.array):
    """Remove all nans from the provided (X, y) pair.
    
    Note: A nan in X means that sample must be removed from *both X and y.
        Likewise, a nan in y means that sample must be removed from *both
        X and y.
    
    :param X: (n, T) array of model inputs
    :param y: (n,) array of labels
    :return: (X, y)
    """
    if not len(X):
        return X, y
    df = pd.DataFrame(columns = list(range(X.shape[1])), data = X)
    df['y'] = y 
    df = df.dropna()
    X = np.array(df.iloc[:, :-1])
    y = np.array(df.iloc[:, -1:].unstack())
    return X, y

answer = time_series_to_numpy(time_series, 10, 2)
answer2 = remove_nans(answer[0], answer[1])

In [ ]:
grader.check("q4ai2")

In [ ]:
def time_series_to_dataset(time_series: pd.DataFrame, T: int, n_val: int):
    """Convert 'time series' dataframe to a numpy dataset.
    
    Uses utilites above `time_series_to_numpy` and `remove_nans`
    
    For description of arguments, see `time_series_to_numpy` docstring.
    """
    convert = time_series_to_numpy(time_series, T, n_val)
    X_train, y_train = remove_nans(convert[0], convert[1])
    X_val, y_val = remove_nans(convert[2], convert[3])
    return X_train, y_train, X_val, y_val
X_train, y_train, X_val, y_val = time_series_to_dataset(time_series_pre, 5, 2)

In [ ]:
grader.check("q4ai3")

In [ ]:
time_series

In [ ]:
time_series_pre

In [ ]:
(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

<!-- BEGIN QUESTION -->

### 4.a.ii. Train and evaluate linear model on pre-lockdown data.

1. **Train a linear model that forecasts the next day's speed average** using your training dataset `X_train`, `y_train`. Specifically, predict $y_{(i,t)}$ from $X_{(i,t)}$, where
- $y_{(i,t)}$ is the daily speed average for day $t$ and census tract $i$
- $X_{(i,t)}$ is a vector of daily speed averages for days $t-5,t-4,t-3,t-2,t-1$ for census tract $i$
2. **Evaluate your model** on your validation dataset `X_val`, `y_val`.
3. **Make a scatter plot**, plotting predicted averages against ground truth averages. Note the perfect model would line up all points along the line $y=x$.

Our model is quantitatively and qualitatively pretty accurate at this point, training and evaluating on pre-lockdown data.

<!--
BEGIN QUESTION
name: q4aii1
points: 1
manual: True
-->

In [ ]:
reg = LinearRegression().fit(X_train, y_train) # set to trained linear model
score = reg.score(X_val, y_val) # report r^2 score
predict = reg.predict(X_val)
# create the scatter plot below
plt.scatter(predict, y_val)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Predicted Averages Against Ground Truth Averages');

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q4aii2
points: 1
-->

In [ ]:
score

In [ ]:
grader.check("q4aii2")

## 4.b. Understand failures on post-lockdown data

Your dataset is distributed spatially and temporally. As a result, the most intuitive spaces to visualize your model error or performance along is both spatially and temporally. In this step, we focus on understanding *where* your model fails.

### 4.b.i. Evaluate on post-lockdown data

1. Using your previously trained linear regression model `reg`, **evaluate on post-lockdown data**, meaning daily speed averages on March 14, 2020. Evaluate on all census tracts.
2. **Make a scatter plot**, plotting predicted averages against ground truth averages. Note the perfect model would line up all points along the line $y=x$.

<!--
BEGIN QUESTION
name: q4bi1
points: 1
-->

In [ ]:
time_series_x_pre = time_series.iloc[:, 8:13].to_numpy() # get 'time series' dataframe for days 8, 10, 11, 12, 13
time_series_y_post = time_series.iloc[:, 13:14].to_numpy() # get 'time series' dataframe for 14th
time_series_x_pre, time_series_y_post = remove_nans(time_series_x_pre, time_series_y_post)
score_pre_14th = reg.score(time_series_x_pre, time_series_y_post)
score_pre_14th

In [ ]:
grader.check("q4bi1")

In [ ]:
time_series_x_pre

In [ ]:
time_series_y_post

<!-- BEGIN QUESTION -->

Make scatter plot below.
<!--
BEGIN QUESTION
name: q4bi2
points: 1
manual: True
-->

In [ ]:
predict = reg.predict(time_series_x_pre)
# create the scatter plot below
plt.scatter(predict, time_series_y_post)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Predicted Averages Against Ground Truth Averages');

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 4.b.ii. Report model performance temporally

1. **Make a line plot** showing performance of the original model throughout all of March 2020.
2. **Report the lowest point on the line plot**, reflecting the lowest model performance.
2. **Why is model performance the worst on the 17th?** Why does it begin to worsen on march 15th? And continue to worsen? Use what you know about covid measures on those dates. You may find this webpage useful: https://abc7news.com/timeline-of-coronavirus-us-covid-19-bay-area-sf/6047519/
3. **Is the dip in performance on the 9th foreshadowed** by any of our EDA?
4. **How does the model miraculously recover on its own?**
5. **Make a scatter plot**, plotting predicted averages against ground truth averages *for model predictions on March 17th*. Note the perfect model would line up all points along the line $y=x$. When compared against previous plots of this nature, this plot looks substantially worse, with points straying far from $y=x$.

**Note:** Answer questions 2-5 in the Markdown cell below. Q1 and Q6 are answered in the two code cells below.

<!--
BEGIN QUESTION
name: q4bii1
points: 3
manual: True
-->



2.The lowest point on the line plot seems to be on the 17th day in March with a model score below 0.75.

3.Given that we know that covid lockdowns began on March 14th, the model performance begins to worsen before March 15th. This is because our model is predicting 5 days prior to the present day. The models performance is the worst on the 17th because the model we are using to predict uses the traffic speeds of the last 5 days and because the shutdown was on the 14ths, 2 of the speeds included were prior to the shutdown, 2 of them were post shutdown and one of them was the exact day of the shutdown. Therefore, this model was a bad predictor of model performance temporally.

4.The dip in performance on the 9th is foreshadowed by our EDA because our prediction is predicting 5 days prior to the actual day. The covid lockdown began on March 14th, and March 9th is 5 days before the actual covid lockdown, which means that our EDA did foreshadow this dip in the model performance.

5.The model miraculously recovers on its own because as we get further in the month and the model uses the 5 day prior method, the 5 days before a later day in the month would use only data from post-lockdown which makes it have a better model performance. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Generate line plot.
<!--
BEGIN QUESTION
name: q4bii2
points: 1
manual: True
-->

In [ ]:
array = []
for i in np.arange(0, 25):
    x_train_b = time_series.iloc[:, i:i+5].to_numpy()
    y_train_b = time_series.iloc[:, i+5:i+6].to_numpy()
    x_train_b, y_train_b = remove_nans(x_train_b, y_train_b)
    array = np.append(array, reg.score(x_train_b, y_train_b))
plt.plot(np.arange(6, 31), array)
plt.xlabel("Day in March")
plt.ylabel("Model Score")
plt.title("Model Score vs. Day in March");

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Generate a scatter plot.
<!--
BEGIN QUESTION
name: q4bii3
points: 1
manual: True
-->

In [ ]:
X, y = remove_nans(time_series.iloc[:, 11:16].to_numpy(), time_series.iloc[:, 16:17].to_numpy())
predict = reg.predict(X)
plt.scatter(x = predict, y = y)
plt.xlabel("Predicted Averages")
plt.ylabel("Ground Truth Averages")
plt.title("Predicted Averages Against Ground Truth Averages");

<!-- END QUESTION -->



## 4.c. "Fix" model on post-lockdown data

Per this survey https://pure.tue.nl/ws/files/3488790/740215.pdf, there are 4 categories of fixes for change points:
- Forgetting mechanisms
- Explicit change detection
- Ensemble techniques
- Context-aware approaches

In this part, we'll combine insights in model errors with previous EDA insights to produce a fix.

<!-- BEGIN QUESTION -->

### 4.c.i. Learn delta off of a moving bias

According to our previous work in EDA, the average speed shoots upwards sharply. As a result, our trick to learn delta the around the average and to naively assume that the average of day $t$ is the average for day $t+1$. We will do this in 4 steps:

1. **Create a dataset for your delta model**.
2. **Train your delta model** on pre-lockdown data.
3. **Evaluate your model on pre-lockdown data**, to ensure that the model has learned to a satisfactory degree, in the nominal case. Remember the naive model achieved 0.97 r^2 on pre-lockdown data.
4. **Evaluate your model on the 17th**, to compare against the naive model also evaluated on that day. Notice that your r^2 score has improved by 10%+. Why is your delta model so effective for the 17th?
5. **Evaluate your model on the 14th**, to compare against the naive model also evaluated on that day. Notice that your r^2 score is now complete garbage. Why is your delta so ineffective for the 14th?

**Hint**: As you build your datasets, always check to make sure you're using the right days! It's easy to have a one-off error that throws off your results.

Write your written questions in the next cell, then write the code in the following cells.

<!--
BEGIN QUESTION
name: q4ci1
points: 2
manual: True
-->

The delta model is so effective for the 17th because we used the 5 day prior model which has the speeds of pre-lockdown, the actual lockdown date and post-lockdown, which was a great way to predict a given day post-lockdown.

The delta model is so ineffective for the 14th because this was the exact day of the lockdown and if we still used the 5 day prior model, the 5 days before the 14ths were all pre-lockdown, so it would not have predicted a massive drop due to the lockdown on that day. In other words, because we used 5 days that were pre-covid lockdown to predict a given date(the given date being the exact day of the shutdown), it would not have predicted this happening.

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q4ci2
points: 1
-->

In [ ]:
time_series_delta = speeds_daily[:13]-time_series_pre  # subtract daily average from pre-lockdown 'time series' dataframe `time_series_pre`
time_series_delta

In [ ]:
grader.check("q4ci2")

In [ ]:
X_delta_train, y_delta_train, X_delta_val, y_delta_val = time_series_to_dataset(time_series_delta, 5, 2)
reg_delta = LinearRegression().fit(X_delta_train, y_delta_train)
res_4ci3 = reg_delta.score(X_delta_val, y_delta_val) # learning delta as easy as learning original dataset!
res_4ci3

In [ ]:
grader.check("q4ci3")

In [ ]:
X, y = remove_nans(time_series.iloc[:, 11:16].to_numpy(), time_series.iloc[:, 16:17].to_numpy())
time_series_x_pre_17 = X - speeds_daily[11:16].array
time_series_17 = y - speeds_daily[17]
res_4ci4 = reg_delta.score(time_series_x_pre_17, time_series_17)
res_4ci4

In [ ]:
grader.check("q4ci4")

In [ ]:
X, y = remove_nans(time_series.iloc[:, 8:13].to_numpy(), time_series.iloc[:, 13:14].to_numpy())
time_series_x_pre_14 = X - speeds_daily[8:13].array
time_series_14 = y - speeds_daily[14]
res_4ci5 = reg_delta.score(time_series_x_pre_14, time_series_14)
res_4ci5

In [ ]:
grader.check("q4ci5")

### 4.c.ii. Does it "solve itself"? Does the pre-lockdown model predict, after the change point?

Had we ignored the problem, would we have been okay? The temporal plot above showing performance over time suggests a partial recovery. **Evaluate the original, naive model on all post-lockdown data** to see. If your final r^2 score does not match the autograder's:
- Double check you have selected daily average speeds for the right days, by printing your dataframe.
- Double check you're using the right model (a brand new trained model)
- Check you're using `T=5, n_val=2`

<!--
BEGIN QUESTION
name: q4cii
points: 1
-->

In [ ]:
X_train_og, y_train_og, X_val_og, y_val_og = time_series_to_dataset(time_series.iloc[:, 13:31], 5, 0)
score_og_post = reg.score(X_train_og, y_train_og)
score_og_post

In [ ]:
grader.check("q4cii")

### 4.c.iii. Naively retrain model with post-lockdown data

Can we use the same tactics--that we used to train the original model on pre-lockdown data--to train on the post-lockdown data? **Retrain a linear model and evaluate on post-lockdown data only**. You should construct a new dataset using `time_series_to_dataset` using only time series from March 14 to March 31. If your final r^2 score does not match the autograder's:
- Double check you have selected daily average speeds for the right days, by printing your dataframe.
- Double check you're using the right model (a brand new trained model)
- Check you're using `T=5, n_val=2`

<!--
BEGIN QUESTION
name: q4ciii
points: 1
-->

In [ ]:
X_train_post, y_train_post, X_val_post, y_val_post = time_series_to_dataset(time_series.iloc[:, 14:31], 5, 0)
score_post = reg.score(X_train_post, y_train_post)
score_post

In [ ]:
grader.check("q4ciii")

### 4.c.iv. What if you just ignore the change point?

Turns out, this is no good. Even acknowledging the change point and training *either* before *or* after is better. Being ignorant and training on *both* is the worst option, producing a lower r^2.

<!--
BEGIN QUESTION
name: q4civ
points: 1
-->

In [ ]:
X_train_iv, y_train_iv, X_val_iv, y_val_iv = time_series_to_dataset(time_series, 5, 5)
reg_iv = LinearRegression().fit(X_train_iv, y_train_iv)
res_4civ = reg_iv.score(X_val_iv, y_val_iv)
res_4civ

In [ ]:
grader.check("q4civ")

# Step 5 - Open-Ended Modeling: Predicting travel time post-lockdown

*This* is the real deal and ultimately what Uber cares about. Traffic speeds is a proxy task, but the bottom line and moneymaking machine relies on this travel time estimation. Focus on designing experiments instead of focusing on experimental, quantitative results. Your experiments are successful if they inform a decision, even despite a lower-performing model.

## Question 5a

Train a baseline model of your choice using any supervised learning approach we have studied; you are not limited to a linear model.


**Example**

Given the data for this question, you could build a model to predict travel time from Cheesecake Factory to UC Berkeley.

In [ ]:
weekdays = [2,3,4,5,6,9,10,11,12,13,16,17,18,19,20,23,24,25,26,27,30,31]
weekday = speeds_to_tract[speeds_to_tract['day'].isin(weekdays)]
weekday.head()

In [ ]:
weekends = [1, 7, 8, 14, 15, 21, 22, 28, 29]
weekend = speeds_to_tract[speeds_to_tract['day'].isin(weekends)]
weekend.head()

In [ ]:
weekday_averages_pre = weekday[weekday['day'] < 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
weekday_averages_pre_named = weekday[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(weekday_averages_pre.to_frame().reset_index())
weekday_averages_pre_named

In [ ]:
weekday_averages_post = weekday[weekday['day'] >= 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
weekday_averages_post_named = weekday[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(weekday_averages_post.to_frame().reset_index())
weekday_averages_post_named

In [ ]:
plt.hist(weekday_averages_pre)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Bay Area Census Tracts Average Speed Pre Lockdown on Weekdays");

In [ ]:
plt.hist(weekday_averages_post)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Bay Area Census Tracts Average Speed Post Lockdown on Weekdays");

In [ ]:
diff_weekday = weekday_averages_pre_named.merge(weekday_averages_post_named, left_on = "DISPLAY_NAME",right_on = "DISPLAY_NAME", how = 'inner')
diff_weekday['differences'] = diff_weekday['speed_mph_mean_y'] - diff_weekday['speed_mph_mean_x']
differences_weekday = diff_weekday['differences']
# plot the differences
plt.hist(differences_weekday);
plt.xlabel("Difference in Average Speed between Pre-Lockdown and Post-Lockdown") 
plt.ylabel("Number of Census Tracts")
plt.title("Differences in Average Speeds Before and After Lockdown on Weekdays");

In [ ]:
weekend_averages_pre = weekend[weekend['day'] < 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
weekend_averages_pre_named = weekend[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(weekend_averages_pre.to_frame().reset_index())
weekend_averages_pre_named

In [ ]:
weekend_averages_post = weekend[weekend['day'] >= 14].groupby('MOVEMENT_ID').agg('mean').sort_values(by ='speed_mph_mean')['speed_mph_mean']
weekend_averages_post_named = weekend[['MOVEMENT_ID', 'DISPLAY_NAME','geometry']].drop_duplicates().merge(weekend_averages_post.to_frame().reset_index())
weekend_averages_post_named

In [ ]:
plt.hist(weekend_averages_pre)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Bay Area Census Tracts Average Speed Post Lockdown on Weekends");

In [ ]:
plt.hist(weekend_averages_post)
plt.xlabel("Average Speed Within Census Tract (mph)")
plt.ylabel("Number of Census Tracts")
plt.title("Bay Area Census Tracts Average Speed Post Lockdown on Weekends");

In [ ]:
diff_weekend = weekend_averages_pre_named.merge(weekend_averages_post_named, left_on = "DISPLAY_NAME",right_on = "DISPLAY_NAME", how = 'inner')
diff_weekend['differences'] = diff_weekend['speed_mph_mean_y'] - diff_weekend['speed_mph_mean_x']
differences_weekend = diff_weekend['differences']
# plot the differences
plt.hist(differences_weekend);
plt.xlabel("Difference in Average Speed between Pre-Lockdown and Post-Lockdown") 
plt.ylabel("Number of Census Tracts")
plt.title("Differences in Average Speeds Before and After Lockdown on Weekends");

In [ ]:
PATH_TIMES = 'data/travel-times-daily-san-francisco-2020-3.csv'

hayes_valley = pd.read_csv(PATH_TIMES)

hayes_valley = hayes_valley.drop(columns = ["Origin Movement ID", "Origin Display Name", "Date Range"])

hayes_valley["Average Bound Difference"] = hayes_valley["Range - Upper Bound Travel Time (Seconds)"] - hayes_valley["Range - Lower Bound Travel Time (Seconds)"] 

hayes_valley.head(10)

In [ ]:
hayes_valley_pre = hayes_valley[hayes_valley["day"] < 14]
hayes_valley_post = hayes_valley[hayes_valley["day"] >= 14]

hayes_valley_pre_grouped = hayes_valley_pre.groupby("Destination Movement ID").agg('mean')
hayes_valley_post_grouped = hayes_valley_post.groupby("Destination Movement ID").agg('mean')

hayes_valley_pre_grouped = hayes_valley_pre_grouped.drop(columns = ["Range - Lower Bound Travel Time (Seconds)", "Range - Upper Bound Travel Time (Seconds)", "day"]).sort_values(by=['Destination Movement ID'])
hayes_valley_post_grouped = hayes_valley_post_grouped.drop(columns = ["Range - Lower Bound Travel Time (Seconds)", "Range - Upper Bound Travel Time (Seconds)", "day"]).sort_values(by=['Destination Movement ID'])
hayes_valley_post_grouped.head()

In [ ]:
pre_removed = hayes_valley_pre_grouped.copy() 
post_removed = hayes_valley_post_grouped.copy() 

pre_tracts = hayes_valley_pre_grouped.index
pre_tracts

post_tracts = hayes_valley_post_grouped.index
post_tracts

remove_these_pre = []
remove_these_post = []

for i in pre_tracts:
    if i not in post_tracts:
        remove_these_pre.append(i)
        
for i in post_tracts:
    if i not in pre_tracts:
        remove_these_post.append(i)

In [ ]:
for i in remove_these_pre:
    pre_removed = pre_removed.drop(i)
    
for i in remove_these_post:
    post_removed = post_removed.drop(i)

In [ ]:
import scipy.stats as st


final = pd.DataFrame()
final.index = pre_removed.index
final["Pre Lockdown Average Bound Size"] = pre_removed["Average Bound Difference"]
final["Post Lockdown Average Bound Size"] = post_removed["Average Bound Difference"]
final["Pre Lockdown Average Travel Time"] = pre_removed["Mean Travel Time (Seconds)"]
final["Post Lockdown Average Travel Time"] = post_removed["Mean Travel Time (Seconds)"]
final["Increase in Average Bound Size Post Lockdown"] = final["Post Lockdown Average Bound Size"] - final["Pre Lockdown Average Bound Size"]

corr_bound_diff_pre = np.corrcoef(final["Increase in Average Bound Size Post Lockdown"], final["Pre Lockdown Average Bound Size"])[1][0]
corr_bound_diff_post = np.corrcoef(final["Increase in Average Bound Size Post Lockdown"], final["Post Lockdown Average Bound Size"])[1][0]

final.plot.scatter(x = "Increase in Average Bound Size Post Lockdown", y = "Post Lockdown Average Bound Size", title = "Increase in Bound Size vs. Average Bound Size (Post Lockdown)");

In [ ]:
time_series_weekdays = time_series.loc[:, weekdays]
time_series_weekends = time_series.loc[:, weekends]

In [ ]:
#Weekday model to predict weekday traffic speeds
X_train_w, y_train_w, X_val_w, y_val_w = time_series_to_dataset(time_series_weekdays, 5, 2)
reg_w = LinearRegression().fit(X_train_w, y_train_w) # set to trained linear model
score_w = reg_w.score(X_val_w, y_val_w) 
predict_w = reg_w.predict(X_val_w)
# create the scatter plot below
plt.scatter(predict_w, y_val_w)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Weekday Model Evaluated on Weekday Data: Predicted Averages Against Ground Truth Averages');
score_w

In [ ]:
#Weekend model to predict weekend traffic speeds
X_train_end, y_train_end, X_val_end, y_val_end = time_series_to_dataset(time_series_weekends, 5, 2)
reg_end = LinearRegression().fit(X_train_end, y_train_end) 
score_end = reg_end.score(X_val_end, y_val_end) 
predict_end = reg_end.predict(X_val_end)
# create the scatter plot below
plt.scatter(predict_end, y_val_end)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Weekend Model Evaluated on Weekend Data: Predicted Averages Against Ground Truth Averages');
score_end

In [ ]:
#Weekday model to predict weekend traffic speeds
X_train_end, y_train_end, X_val_end, y_val_end = time_series_to_dataset(time_series_weekends, 5, 2)
score_week = reg_w.score(X_val_end, y_val_end) 
predict_week = reg_w.predict(X_val_end)
# create the scatter plot below
plt.scatter(predict_week, y_val_end)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Weekday Model Evaluated on Weekend Data: Predicted Averages Against Ground Truth Averages');
score_week

In [ ]:
#Weekend model to predict weekday traffic speeds
X_train_w, y_train_w, X_val_w, y_val_w = time_series_to_dataset(time_series_weekdays, 5, 2)
score_weeks = reg_end.score(X_val_w, y_val_w) 
predict_weeks = reg_end.predict(X_val_w)
# create the scatter plot below
plt.scatter(predict_weeks, y_val_w)
plt.xlabel('Predicted Averages')
plt.ylabel('Ground Truth Averages')
plt.title('Weekend Model Evaluated on Weekday Data: Predicted Averages Against Ground Truth Averages');
score_weeks

## Question 5b

Improve on your baseline model. Specify the model you designed and its input features. Justify why you chose these features and their relevance to your model's predictions.

**Example**

Here are potential questions to consider for this part: How does the other variant of your travel times dataset, aggregated across time but reported for all routes, useful?  What additional data from the Uber Movement website can you export to better your model?

## Question 5c

Explore other modeling aspects and/or temporal information. You are free to relate this to your hypothesis or not. Please expand into multiple parts that logically separate and break down your modeling work!

**Example**

For example, explore change across time, before and after the lockdown: (a) train and evaluate on *pre*-lockdown traffic travel times for that route; and (b) evaluate your model on *post*-lockdown traffic patterns.
How would you correct your model for a more accurate post-lockdown traffic predictor? *The above is just a suggestion. You may pick any topic you find interesting.*

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()